# Log/Crash Parsing & Simple Anomaly Detection

Find crash spikes using a rolling z‑score.

In [ ]:
import pandas as pd, numpy as np
from datetime import datetime
import matplotlib.pyplot as plt

df = pd.read_csv('../data/synthetic_telemetry.csv', parse_dates=['ts'])
cr = df[df['event']=='crash'].copy()
if cr.empty:
    print("No crash events in this synthetic dataset.")
else:
    cr['date'] = cr['ts'].dt.date
    daily = cr.groupby('date').size().rename('crash_count').reset_index()
    daily['rolling_mean'] = daily['crash_count'].rolling(7, min_periods=3).mean()
    daily['rolling_std'] = daily['crash_count'].rolling(7, min_periods=3).std()
    daily['z'] = (daily['crash_count'] - daily['rolling_mean'])/daily['rolling_std']

    plt.figure()
    plt.plot(daily['date'], daily['crash_count'])
    plt.title('Daily Crash Count')
    plt.xlabel('Date'); plt.ylabel('Crashes')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

    # Flag anomalies where z > 2
    anomalies = daily[daily['z']>2]
    print("Anomalies (z>2):")
    print(anomalies[['date','crash_count','z']])